In [88]:
import nbimporter
import requests
import os
import json
from bs4 import BeautifulSoup
from openai import OpenAI
from dotenv import load_dotenv
from common import Website
from IPython.display import Markdown, display, update_display

In [83]:
load_dotenv()

openapi_key = os.getenv("OPENAI_API_KEY")

MODEL="gpt-4o-mini"

openai = OpenAI()

In [68]:
system_prompt = "You are an assistant that can choose relevant links for a brochure of a company from the given list of links of a website.\n"
system_prompt += "Give response in json format as below: \n" 
system_prompt +=              """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

print (system_prompt)

You are an assistant that can choose relevant links for a brochure of a company from the given list of links of a website.
Give response in json format as below: 

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [69]:
def get_user_prompt():
    user_prompt = "Here are the links of a website"
    user_prompt += "\n".join(web.links)
    user_prompt += "Please decide which links are relevant to create brocure for the company. Skip pages like Terms of Service, Privacy. Respond with the full https URL in JSON format"
    return user_prompt

In [75]:
def get_links(url):

    web = Website(url)
    user_prompt = get_user_prompt()
    openai = OpenAI()
    messages = [{"role" : "system", "content": system_prompt},{"role" : "user" , "content": user_prompt}]
    response = openai.chat.completions.create(model=MODEL, messages = messages)
    links = response.choices[0].message.content
    return json.loads(links)

{'links': [{'type': 'about page', 'url': 'https://www.explodingkittens.com/pages/about'}, {'type': 'contact page', 'url': 'https://www.explodingkittens.com/pages/contact'}, {'type': 'blog', 'url': 'https://www.explodingkittens.com/blog'}, {'type': 'products page', 'url': 'https://www.explodingkittens.com/products/throw-throw-burrito-original-edition'}, {'type': 'animations', 'url': 'https://www.explodingkittens.com/animations'}, {'type': 'comics', 'url': 'https://www.explodingkittens.com/comics'}, {'type': 'games', 'url': 'https://www.explodingkittens.com/games'}, {'type': 'books', 'url': 'https://www.explodingkittens.com/books'}]}


In [78]:
#get contents of relevant pages of website
def get_website_summary(url):
    summary = Website(url).get_contents()
    links = get_links(url)
    
    for link in links["links"]:
        link_contents = Website(link["url"]).get_contents()
        summary += link_contents
    return summary


Webpage Title:
The Oatmeal - Comics by Matthew Inman 
Webpage Contents:
The Oatmeal
Home
Comics
Animations
(NEW)
Games
Books
Subscribe
Blog
Hi
. My name is Matthew Inman. I'm a cartoonist.
I've been publishing comics on this website since 2009. 
				Every few years, I compile these comics into books, such as
How to tell if your cat is plotting to kill you.
I also make card games, such as
Exploding Kittens
and
Throw Throw Burrito.
Also, my Netflix series
is now streaming.
The Latest
1
|
2
|
3
|
4
|
5
|
6
|
7
|
8
|
9
|
Next Page »
Popular comics
Home
Comics
Games
Books
Blog
Quizzes
About
Contact
RSS
-
Mastodon
-

Webpage Title:
404 Not Found
Webpage Contents:
Skip to content
Previous
Free shipping on orders of $40 or more in the contiguous US.
See Details.
Close
US FREE SHIPPING Terms & Conditions
Offer valid on order with subtotals of $40 or more. Tax excluded. Discount codes may change your subtotal. Offer not redeemable for cash or credit. Exploding Kittens reserves the right to modif

In [54]:
brochure_system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

def get_brochure_user_prompt(url):
    brochure_user_prompt = f"You are looking at the website {url} \n. Please create a short brochure for the company. \nHere is the content of several relevant pages of a website \n"
    brochure_user_prompt += get_website_summary(url)
    return brochure_user_prompt
    

In [80]:
url = "https://theoatmeal.com/"

print(get_brochure_user_prompt(url))

You are looking at the website https://theoatmeal.com/ 
. Please create a short brochure for the company. 
Here is the content of several relevant pages of a website 
Webpage Title:
The Oatmeal - Comics by Matthew Inman 
Webpage Contents:
The Oatmeal
Home
Comics
Animations
(NEW)
Games
Books
Subscribe
Blog
Hi
. My name is Matthew Inman. I'm a cartoonist.
I've been publishing comics on this website since 2009. 
				Every few years, I compile these comics into books, such as
How to tell if your cat is plotting to kill you.
I also make card games, such as
Exploding Kittens
and
Throw Throw Burrito.
Also, my Netflix series
is now streaming.
The Latest
1
|
2
|
3
|
4
|
5
|
6
|
7
|
8
|
9
|
Next Page »
Popular comics
Home
Comics
Games
Books
Blog
Quizzes
About
Contact
RSS
-
Mastodon
-

Webpage Title:
404 Not Found
Webpage Contents:
Skip to content
Previous
Free shipping on orders of $40 or more in the contiguous US.
See Details.
Close
US FREE SHIPPING Terms & Conditions
Offer valid on order with 

In [91]:
messages = [{"role": "system", "content": brochure_system_prompt}, {"role": "user", "content": brochure_user_prompt}]
response = openai.chat.completions.create(model=MODEL, messages=messages)
result = response.choices[0].message.content
display(Markdown(result))

# The Oatmeal Company Brochure

![The Oatmeal](https://theoatmeal.com/assets/images/the-oatmeal-header.png)

## About Us
The Oatmeal is a unique blend of humor and learning, created by Matthew Inman. As an animated webcomic, we aim to entertain and educate our audience by tackling complex subjects, from grammar to science, with a light-hearted and relatable twist. Our mission is to spread knowledge and joy through engaging content that resonates with readers of all ages.

## Our Culture
At The Oatmeal, we foster a creative and inclusive environment where ideas can flourish. Our team embraces a work hard, play hard philosophy, ensuring that while we’re dedicated to delivering quality content, we also carve out time for laughter and camaraderie. Collaboration is at the heart of our process, and every member is encouraged to contribute their unique voice.

### Core Values
- **Creativity**: We believe in the power of imagination and strive for innovative approaches in our content creation.
- **Humor**: Laughter is a universal language, and we utilize it to connect with our audience on various topics.
- **Education**: Our goal is to make learning enjoyable by demystifying complex concepts.

## Our Customers
The Oatmeal caters to a diverse audience that includes students, educators, parents, and anyone looking to add a dash of humor to their learning. Our readers appreciate the blend of fun, wit, and valuable information that our content provides. We engage our audience through various channels, from illustrated comics to interactive quizzes and educational materials.

## Careers at The Oatmeal
Joining The Oatmeal team means being part of a passionate group dedicated to making knowledge engaging and accessible. We’re always on the lookout for talented individuals who share our values and wants to contribute to a creative community. Opportunities may range from content creation and graphic design to marketing and community engagement.

### Why Work Here?
- **Creative Freedom**: We support out-of-the-box thinking and allow our team to explore their creativity.
- **Flexible Work Arrangements**: We understand the importance of work-life balance and offer flexible hours.
- **Growth Opportunities**: We provide an environment that encourages professional development and career advancement.

## Join Us!
Whether you are looking to explore our humor-filled comics, invest in a unique educational platform, or find a fulfilling career, The Oatmeal welcomes you! Experience the joy of learning and creativity with us today!

### Connect With Us
- **Website**: [theoatmeal.com](https://theoatmeal.com)
- **Twitter**: [@theoatmeal](https://twitter.com/theoatmeal)
- **Instagram**: [@theoatmeal](https://instagram.com/theoatmeal)

---
*Thank you for considering The Oatmeal. We look forward to embarking on a vibrant journey of laughter and learning with you!*